In [1]:
####
#Author: yuehao wang
#version 1.0
#references: 
#https://www.programiz.com/python-programming/working-csv-files
#https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.create_bucket
#https://realpython.com/python-boto3-aws-s3/
#CLI aws s3api create-bucket --bucket my-bucket-name --region us-west-2 --create-bucket-configuration LocationConstraint=us-west-2
#https://robertorocha.info/setting-up-a-selenium-web-scraper-on-aws-lambda-with-python/ 
##

###Load libraries
#import awscli
import selenium
import boto3
import pandas as pd
import time
import s3fs

from selenium import webdriver


####SCRAPE THE WEBSITE######
###call the webdriver
browser = webdriver.Chrome("D:\dev\chromedriver.exe")

#enter the url path that needs to be accessed by webdriver
browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities.jsp')

#identify xpath of location to select element
inputElement = browser.find_element_by_xpath("/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]")
inputElement.send_keys('0')
inputElement1 = browser.find_element_by_xpath("/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]").click()

#identify the table to scrape
table = browser.find_element_by_css_selector('table.Bordered')

print(table)

<selenium.webdriver.remote.webelement.WebElement (session="7cb3c69d45a5da2912d4969bc7dd52c9", element="73030ca3-5032-4c9e-aade-4c98656eceae")>


### The html of table is followed:

```
  <table cellpadding="4" class="Bordered">
    <thead>
        <tr>
        <th class="sortable sorted order1">Organization Name</th>
        </tr>
    </thead>
    
    <tbody>
        <tr>
        <td>"Studio 5404" Inc.</td>
        </tr>
    </tbody>
```

### Step 6. Note that the output of the file has an issue! The header inserts a blank row! Update the script that was provided to remove the blank row in the csv output file

### we should select tbody first, then to select  td

- tbody = table.find_element_by_css_selector('tbody')
- for row in tbody.find_elements_by_css_selector('tr'):

In [2]:
#####CREATE DATE FRAME#####
#create empty dataframe
df =[]

# we only select td list in the tbody
tbody = table.find_element_by_css_selector('tbody')
    
#loop through dataframe to export table
for row in tbody.find_elements_by_css_selector('tr'):
      cols = df.append([cell.text for cell in row.find_elements_by_css_selector('td')])


#update dataframe with header 
df = pd.DataFrame(df, columns = ["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"])
display(df) #let's have a look at the data before creating the CSV file and loading it into s3

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
1,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
2,(ASMA) American Syrian Multicultural Associati...,42-84-63,273130182,NFP,BROOKLYN,NY
3,#HicksStrong Inc.,48-10-48,842612081,NFP,CLIFTON PARK,NY
4,#WalkAway Foundation,47-15-80,832820906,NFP,CARLSBAD,CA
5,04/11 10:17 PM test,47-13-95,206256427,NFP,ALBANY,NY
6,1/20/21 Action Fund,46-99-13,832210730,NFP,SAN FRANCISCO,CA
7,"10/40 Connections, Inc.",45-70-15,621825230,NFP,HIXSON,TN
8,"1000 Feet Project, Inc",45-00-14,473820859,NFP,NEW YORK,NY
9,1000 Islands Hose Haulers,45-38-38,454570241,NFP,CARTHAGE,NY


In [3]:
def save_to_s3(df):
    
    # df.to_csv(None, index=False) can remove the blank header
    byte_encoded_csv = df.to_csv(None, index=False).encode() #encodes file as binary
    
    # s3://g2-final/readme.txt
    pathname = 's3://yw-m10-database-update-bucket/' #specify location of s3:/{my-bucket}/
    filename= 'yuehao_wang_m10_' #name of your group
    datetime = time.strftime("%Y%m%d%H%M%S") #timestamp
    
    filenames3 = "%s%s%s.csv"%(pathname, filename, datetime) #name of the filepath and csv file
    print(filenames3)
    
    s3 = s3fs.S3FileSystem(anon=False)
    
    with s3.open(filenames3, 'wb') as file:
        file.write(byte_encoded_csv) #writes byte-encoded file to s3 location

    #print success message
    print("Successfull uploaded file to location:"+str(filenames3))
    
    return filenames3

# remove the first row, because it is all None
s3_filename = save_to_s3(df)

s3://yw-m10-database-update-bucket/yuehao_wang_m10_20210403211319.csv
Successfull uploaded file to location:s3://yw-m10-database-update-bucket/yuehao_wang_m10_20210403211319.csv


### Step 7. Test the file by inspecting the output on the s3 bucket

In [4]:
# test the csv. download from s3, than print it
test_df = pd.read_csv(str(s3_filename))
test_df.head(2)

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
1,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY


### save s3 csv file to local

In [5]:
save_path = "C:/Users/amorn/Documents/yeshiva/workspace/M10_Assignment/files/"+str(s3_filename[35:])
test_df.to_csv(save_path)

# Part II. Update web-scraper to iterate all results and load csv file into S3 Bucket

#### 1) select the pagelinks by class name

In [6]:
pages = browser.find_elements_by_xpath("//a[starts-with(@title,'Go to page')]")

page_links = [] # save all page links into page_links
for page in pages:
    
    print(page.get_attribute("href"))

    page_links.append(page.get_attribute("href"))

    

https://www.charitiesnys.com/RegistrySearch/search_charities_action.jsp?orgName=&d-49653-p=2&city=&searchType=contains&reg1=&project=Charities&reg3=&reg2=&ein=0-&orgId=&num1=0&state=none&regType=ALL&num2=
https://www.charitiesnys.com/RegistrySearch/search_charities_action.jsp?orgName=&d-49653-p=3&city=&searchType=contains&reg1=&project=Charities&reg3=&reg2=&ein=0-&orgId=&num1=0&state=none&regType=ALL&num2=
https://www.charitiesnys.com/RegistrySearch/search_charities_action.jsp?orgName=&d-49653-p=4&city=&searchType=contains&reg1=&project=Charities&reg3=&reg2=&ein=0-&orgId=&num1=0&state=none&regType=ALL&num2=
https://www.charitiesnys.com/RegistrySearch/search_charities_action.jsp?orgName=&d-49653-p=5&city=&searchType=contains&reg1=&project=Charities&reg3=&reg2=&ein=0-&orgId=&num1=0&state=none&regType=ALL&num2=
https://www.charitiesnys.com/RegistrySearch/search_charities_action.jsp?orgName=&d-49653-p=6&city=&searchType=contains&reg1=&project=Charities&reg3=&reg2=&ein=0-&orgId=&num1=0&stat

#### 2) loop the page_links, open each links, scrape the table, put table into dataframe

In [14]:
df_list = []
df_list.append(df)

# loop the page_links
for page_link in page_links:
    
    # open link
    print(page_link)
    page_tab = browser.get(page_link)
    
    # identify the table to scrape
    table = browser.find_element_by_css_selector('table.Bordered')
    
    # create empty dataframe
    page_df =[]

    # loop through dataframe to export table
    #for row in table.find_elements_by_css_selector('tr'):
    #      cols = page_df.append([cell.text for cell in row.find_elements_by_css_selector('td')])
    
    # we only select td list in the tbody
    tbody = table.find_element_by_css_selector('tbody')

    #loop through dataframe to export table
    for row in tbody.find_elements_by_css_selector('tr'):
          cols = page_df.append([cell.text for cell in row.find_elements_by_css_selector('td')])


    # update dataframe with header 
    page_df = pd.DataFrame(page_df, columns = ["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"])
    # display(page_df) #let's have a look at the data before creating the CSV file and loading it into s3
    
    df_list.append(page_df)
    
    # sleep 2s
    time.sleep(2)
    

# concat all df_list as one dataframe
df_all = pd.concat(df_list)
df_all

https://www.charitiesnys.com/RegistrySearch/search_charities_action.jsp?orgName=&d-49653-p=2&city=&searchType=contains&reg1=&project=Charities&reg3=&reg2=&ein=0-&orgId=&num1=0&state=none&regType=ALL&num2=
https://www.charitiesnys.com/RegistrySearch/search_charities_action.jsp?orgName=&d-49653-p=3&city=&searchType=contains&reg1=&project=Charities&reg3=&reg2=&ein=0-&orgId=&num1=0&state=none&regType=ALL&num2=
https://www.charitiesnys.com/RegistrySearch/search_charities_action.jsp?orgName=&d-49653-p=4&city=&searchType=contains&reg1=&project=Charities&reg3=&reg2=&ein=0-&orgId=&num1=0&state=none&regType=ALL&num2=
https://www.charitiesnys.com/RegistrySearch/search_charities_action.jsp?orgName=&d-49653-p=5&city=&searchType=contains&reg1=&project=Charities&reg3=&reg2=&ein=0-&orgId=&num1=0&state=none&regType=ALL&num2=
https://www.charitiesnys.com/RegistrySearch/search_charities_action.jsp?orgName=&d-49653-p=6&city=&searchType=contains&reg1=&project=Charities&reg3=&reg2=&ein=0-&orgId=&num1=0&stat

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
1,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
2,(ASMA) American Syrian Multicultural Associati...,42-84-63,273130182,NFP,BROOKLYN,NY
3,#HicksStrong Inc.,48-10-48,842612081,NFP,CLIFTON PARK,NY
4,#WalkAway Foundation,47-15-80,832820906,NFP,CARLSBAD,CA
...,...,...,...,...,...,...
5,University of Virginia Health Foundtion,40-44-88,412097394,NFP,CHARLOTTESVILLE,VA
6,Violin Player,41-40-19,270773158,NFP,EAST AMHERST,NY
7,"William A. Epps Community Center, Inc.",40-91-11,861074714,NFP,STATEN ISLAND,NY
8,WORLD SOCIETY OF CZESTOCHOWA JEWS AND THEIR DE...,40-46-49,205101779,NFP,NEW YORK,NY


In [15]:
# sent to s3
s3_filename = save_to_s3(df_all)

s3://yw-m10-database-update-bucket/yuehao_wang_m10_20210403212150.csv
Successfull uploaded file to location:s3://yw-m10-database-update-bucket/yuehao_wang_m10_20210403212150.csv


In [16]:
# test the csv. download from s3, than print it
test_df = pd.read_csv(str(s3_filename))
test_df.head(2)

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
1,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY


In [17]:
test_df.shape

(100, 6)

### save s3 csv file to local

In [18]:
save_path = "C:/Users/amorn/Documents/yeshiva/workspace/M10_Assignment/files/"+str(s3_filename[35:])
test_df.to_csv(save_path)